In [ ]:
import gym
import numpy
import tensorflow as tf
from collections import deque

env = gym.make('CartPole-v1')
env.reset()
for _ in range(10000):
    env.render()
    env.step(env.action_space.sample()) # take a random action
env.close()

In [ ]:
import gym
env = gym.make('Breakout-v0')
for i_episode in range(20):
    observation = env.reset()
    for t in range(1000):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

**CartPole example**

**Initialization**

In [1]:
import gym
env = gym.make('CartPole-v1')
print(env.action_space)

Discrete(2)


In [2]:
print(env.observation_space)

Box(4,)


In [3]:
print(env.observation_space.high)

[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


In [4]:
print(env.observation_space.low)

[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]


https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [5]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from collections import deque
env = gym.make("CartPole-v1")
input_shape = [4] #=env.observation_space.shape
n_outputs = 2 # ==env.action_space.n

model = keras.Sequential([
    keras.layers.Dense(32, activation="relu", input_shape=input_shape),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(n_outputs)
])

**Definition of the epsilon-greedy policy**

In [6]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(2)
    else:
        Q_values = model.predict(state[np.newaxis])
        return np.argmax(Q_values[0])

**Creation of the replay buffer**

In [7]:
replay_buffer = deque(maxlen=2000)

**Sampling experience**

In [8]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    states, actions, rewards, next_states, dones = [np.array([experience[field_index] for experience in batch])
    for field_index in range(5)]
    return states, actions, rewards, next_states, dones

**Network initialization**

In [9]:
batch_size = 32
discount_factor = 0.95
optimizer = keras.optimizers.Adam(lr=1e-3)
loss_fn = keras.losses.mean_squared_error

**Training initialization**

In [10]:
def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states, dones = experiences
    next_Q_values = model.predict(next_states)
    max_next_Q_values = np.max(next_Q_values, axis=1)
    target_Q_values = (rewards + (1-dones)*discount_factor*max_next_Q_values)
    mask = tf.one_hot(actions, n_outputs)
    with tf.GradientTape() as tape:
        all_Q_values = model(states)
        Q_values = tf.reduce_sum(all_Q_values*mask, axis=1, keepdims=True)
        loss = tf.reduce_mean(loss_fn(target_Q_values,Q_values))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

**Evaluation**

In [11]:
def play_one_step(env, state, epsilon):
    action = epsilon_greedy_policy(state, epsilon)
    next_state, reward, done, info = env.step(action)
    replay_buffer.append((state, action, reward, next_state, done))
    return next_state, reward, done, info

In [12]:
for episode in range(600):
    obs = env.reset()
    for step in range(200):
        epsilon = max(1- episode/500, 0.01)
        obs, reward, done, info = play_one_step(env, obs, epsilon)
        env.render()
        if done:
            break
        if episode>50:
            training_step(batch_size)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

